# Torch X-Ray Vision

https://github.com/mlmed/torchxrayvision

In [6]:
import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import sys
import torch.nn.functional as F
import tqdm
import sklearn, sklearn.metrics
import pandas as pd
import torchxrayvision as xrv
import skimage.io

In [7]:
# Prepare the image:
img = skimage.io.imread("16747_3_1.jpg")
img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
img = img.mean(2)[None, ...] # Make single color channel

transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

img = transform(img)
img = torch.from_numpy(img)

# Load model and process image
model = xrv.models.DenseNet(weights="densenet121-res224-all")
outputs = model(img[None,...]) # or model.features(img[None,...]) 

# Print results
dict(zip(model.pathologies,outputs[0].detach().numpy()))

FileNotFoundError: No such file: '/cluster/home/taheeraa/code/forprosjekt/16747_3_1.jpg'

In [13]:
d_nih = xrv.datasets.NIH_Dataset(imgpath="cluster/home/taheeraa/data/images-512-NIH")
xrv.datasets.relabel_dataset(xrv.datasets.default_pathologies, d_nih)

Exception: imgpath must be a directory

In [ ]:
d_pc = xrv.datasets.PC_Dataset(imgpath="/home/users/taheeraa/data/images-512-NIH")
xrv.datasets.relabel_dataset(xrv.datasets.default_pathologies, d_pc)

In [ ]:
sample = d_nih[40]
plt.imshow(sample["img"][0], cmap="Greys_r");
dict(zip(d_pc.pathologies,sample["lab"]))

In [ ]:
model = xrv.models.DenseNet(weights="all")
# does the model align with the dataset?
dict(zip(model.pathologies,xrv.datasets.default_pathologies))


In [ ]:
model = xrv.models.DenseNet(weights="all")
with torch.no_grad():
    out = model(torch.from_numpy(sample["img"]).unsqueeze(0)).cpu()
    
dict(zip(model.pathologies,zip(out[0].detach().numpy(), sample["lab"])))

In [ ]:
model = xrv.models.DenseNet(weights="nih")
with torch.no_grad():
    out = model(torch.from_numpy(sample["img"]).unsqueeze(0)).cpu()
    
dict(zip(model.pathologies,zip(out[0].detach().numpy(), sample["lab"])))

In [ ]:
# small test eval
model = xrv.models.DenseNet(weights="all")
if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
outs = []
labs = []
with torch.no_grad():
    np.random.seed(0)
    for i in tqdm.tqdm(np.random.randint(0,len(d_nih),500)):
        sample = d_nih[i]
        labs.append(sample["lab"])
        image = torch.from_numpy(sample["img"]).unsqueeze(0)
        if torch.cuda.is_available():
            image = image.cuda()
        out = model(image).cpu()
        outs.append(out.detach().numpy()[0])


In [ ]:
results = []
for i in range(14):
    result = {}
    result["Pathology"] = model.pathologies[i]
    if len(np.unique(np.asarray(labs)[:,i])) > 1:
        labels = np.asarray(labs)[:,i].astype(bool)
        preds = np.asarray(outs)[:,i]
        result["AUC"] = sklearn.metrics.roc_auc_score(labels, preds)
        result["Acc"] = sklearn.metrics.accuracy_score(labels, preds > 0.5)
        result["F1"] = sklearn.metrics.f1_score(labels, preds > 0.5)
    results.append(result)

In [ ]:
pd.DataFrame(results)